In [1]:
pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [39]:
import pyautogui
import time
from PIL import ImageGrab
import copy
import random
BOX_LEN=24
WIDTH=30
HEIGHT=20
mine_map=list()

In [40]:
def distance(x,y):
    return (x[0]-y[0])**2+(x[1]-y[1])**2+(x[2]-y[2])**2

def check_color(pixel_position,screen):
    global COLOR
    pixel_col=screen.getpixel(pixel_position)
    now_color = COLOR.get(pixel_col)
    if now_color != None:
        if now_color==0:
            new_position=pyautogui.Point(pixel_position[0]-11,pixel_position[1]-11)
            new_color=screen.getpixel(new_position)
            if distance(new_color,(255,255,255)) <= 100:
                return 0
            else:
                new_position=pyautogui.Point(pixel_position[0]+4,pixel_position[1])
                new_color=screen.getpixel(new_position)
                
                return 9
        else :
            return now_color
    else:
        for color in COLOR:
            #print(distance(color,pixel_col),pixel_col)
            if distance(color,pixel_col)<=200:
                return COLOR[color]
        return None

#time.sleep(3)
#screen = ImageGrab.grab()
#x=25
#y=0
#check_point=pyautogui.Point(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
#print(check_color(check_point,screen))

In [41]:
def range_list(x,y,i):
    if x==0:
        range_x=range(0,1+i)
    elif x==WIDTH-1:
        range_x=range(WIDTH-1-i,WIDTH)
    else:
        range_x=range(x-i,x+1+i)
        
    if y==0:
        range_y=range(0,1+i)
    elif y==HEIGHT-1:
        range_y=range(HEIGHT-1-i,HEIGHT)
    else:
        range_y=range(y-i,y+1+i)
    
    return range_x,range_y


def check_surround_mine(x,y):
    global mine_map
    global start_x
    global start_y
    
    range_x,range_y=range_list(x,y)
        
    sum=0
    try:
        for i in range_x:
            for j in range_y:
                if mine_map[j][i]==0:
                    sum+=1
        if sum==mine_map[y][x]:
            #pyautogui.click(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
            #print(x,y)
            return True
        else:
            return False
    except:
        print("error x,y : ")
        print(x,y)

In [42]:
def find_blank(x,y,size):
    global mine_map
    surrounded = list()
    check_surrounded=list()
    range_x,range_y=range_list(x,y,1)
    
    check_surrounded.append([x,y])
    
    for i in range_x:
        for j in range_y:
            if mine_map[j][i]>0 and mine_map[j][i]<9 :
                if not [i,j] in surrounded:
                    surrounded.append([i,j])
    
    for k in range(size):
        for sur in surrounded:
            range_x,range_y=range_list(sur[0],sur[1],1)
            for i in range_x:
                for j in range_y:
                    if mine_map[j][i]==0:
                        if not [i,j] in check_surrounded:
                            check_surrounded.append([i,j])

        for che in check_surrounded:
            range_x,range_y=range_list(che[0],che[1],1)
            for i in range_x:
                for j in range_y:
                    if mine_map[j][i]>0 and mine_map[j][i]<9 :
                        if not [i,j] in surrounded:
                            surrounded.append([i,j])
    
    return check_surrounded, surrounded

In [44]:
def find_mine(x,y,size,remain_mine):
    global remain_blank
    
    if remain_blank<13:
        check_mine_list=list()
        check_mine_list.append([x,y])
        surrounded_list=list()
        for i in range(WIDTH):
            for j in range(HEIGHT):
                if mine_map[j][i]==0 and not [i,j] in check_mine_list:
                    check_mine_list.append([i,j])
        for che in check_mine_list:
            range_x,range_y=range_list(che[0],che[1],1)
            for i in range_x:
                for j in range_y:
                    if mine_map[j][i]>0 and mine_map[j][i]<9 :
                        if not [i,j] in surrounded_list:
                            surrounded_list.append([i,j])
                            
    else:
        check_mine_list, surrounded_list = find_blank(x,y,size)
    pos = dfs(check_mine_list,surrounded_list,1,1,remain_mine)
    
    #mine
    if pos==0:
        return 0
    
    impos = dfs(check_mine_list,surrounded_list,1,0,remain_mine)
    
    #not mine
    if impos==0:
        return 1
    
    #don't know
    else:
        return 2
    
def dfs(check_mine_list,surrounded_list,now,seq,remain_mine):
    global mine_map
    global remain_blank
    return_val = 0
    mine_count=0
    
    if now==(len(check_mine_list)):
        if remain_blank<13:
            for bit in range(remain_blank):
                if (1<<bit)&seq!=0:
                    mine_count+=1
            if mine_count!=remain_mine:
                return 0
        for sur in surrounded_list:
            mine_count=0
            isblank=False
            range_x,range_y=range_list(sur[0],sur[1],1)
            for x in range_x:
                for y in range_y:
                    if mine_map[y][x]==-1:
                        mine_count+=1
                    elif [x,y] in check_mine_list:
                        if (seq & (1<<check_mine_list.index([x,y])))!=0:
                            mine_count+=1
                    elif mine_map[y][x]==0:
                        isblank=True
                        
            if isblank:
                if mine_count>mine_map[sur[1]][sur[0]]:
                    return 0
            else:
                if mine_count!=mine_map[sur[1]][sur[0]]:
                    return 0
        return 1
        
    if not check_condition(check_mine_list,surrounded_list,now,seq):
        return 0
    
    return_val = return_val | dfs(check_mine_list,surrounded_list,now+1,seq ,remain_mine)
    return_val = return_val | dfs(check_mine_list,surrounded_list,now+1,(1<<now)| seq ,remain_mine)
    
    return return_val

def check_condition(check_mine_list,surrounded_list,now,seq):
    for sur in surrounded_list:
        mine_count=0
        isblank=False
        range_x,range_y=range_list(sur[0],sur[1],1)
        for x in range_x:
            for y in range_y:
                if mine_map[y][x]==-1:
                    mine_count+=1
                elif [x,y] in check_mine_list and check_mine_list.index([x,y])<now:
                    if (seq & (1<<check_mine_list.index([x,y])))!=0:
                        mine_count+=1
                elif mine_map[y][x]==0:
                    isblank=True

        if isblank:
            if mine_count>mine_map[sur[1]][sur[0]]:
                return 0
        else:
            if mine_count!=mine_map[sur[1]][sur[0]]:
                return 0
    return 1
'''
for x in range(0,30):
    for y in range(0,20):
        if mine_map[y][x]==0:
            print("x : {0}, y: {1}, {2}".format(x,y,find_mine(x,y,1,130)))
print('done')
'''
'''
remain_blank=4
for x in range(0,WIDTH):
    for y in range(0,HEIGHT):
        if mine_map[y][x]==0:
            print("x : {0}, y: {1}, {2}".format(x,y,find_mine(x,y,1,3)))
'''

'\nremain_blank=4\nfor x in range(0,WIDTH):\n    for y in range(0,HEIGHT):\n        if mine_map[y][x]==0:\n            print("x : {0}, y: {1}, {2}".format(x,y,find_mine(x,y,1,3)))\n'

In [55]:
#find first position
time.sleep(WAIT)
buttonlocation = pyautogui.locateOnScreen('start.png',confidence=.99)
startpoint=pyautogui.center(buttonlocation)
start_x=startpoint.x-BOX_LEN*((WIDTH-1)/2)
start_y=startpoint.y+52
screen = ImageGrab.grab()

#initalize mine_map array
for x in range(WIDTH):
    for y in range(HEIGHT):
        check_point=pyautogui.Point(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
        mine_map[y][x] = check_color(check_point,screen)

for y in range(HEIGHT):
    print(mine_map[y])


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 9, 9, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 9, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [18]:
def game_auto_clear(remain_mine,find_mine_num):
    global remain_blank
    time.sleep(WAIT)
    buttonlocation = pyautogui.locateOnScreen('start.png',confidence=.99)
    startpoint=pyautogui.center(buttonlocation)
    start_x=startpoint.x-BOX_LEN*((WIDTH-1)/2)
    start_y=startpoint.y+52
    screen = ImageGrab.grab()
    sx=-1
    sy=-1

    #initalize mine_map array
    for x in range(WIDTH):
        for y in range(HEIGHT):
            check_point=pyautogui.Point(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
            mine_map[y][x] = check_color(check_point,screen)
            if mine_map[y][x] is None:
                sx=x
                sy=y
                
    if sx!=-1 and sy!=-1:
        pyautogui.click(start_x+BOX_LEN*sx,start_y+BOX_LEN*sy)

    for l in range(find_mine_num):
        time.sleep(WAIT)
        print("remain_mine "+str(remain_mine))
        print("remain_blank "+str(remain_blank))
        screen = ImageGrab.grab()
        for x in range(WIDTH):
            for y in range(HEIGHT):
                if mine_map[y][x]==0 or mine_map[y][x]==9:
                    check_point=pyautogui.Point(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
                    mine_map[y][x] = check_color(check_point,screen)
                    if mine_map[y][x] is None:
                        mine_map[y][x]=9
        remain_blank=0                
        for x in range(WIDTH):
            for y in range(HEIGHT):
                if mine_map[y][x]==0:
                    remain_blank+=1
                    
        if remain_blank==0:
            return
        
        isfind=False
        for k in range(1,6,2):
            for x in range(0,WIDTH):
                for y in range(0,HEIGHT):
                    if mine_map[y][x]==0:
                        value = find_mine(x,y,k,remain_mine)
                        if value==1:
                            mine_map[y][x]=-1
                            remain_mine-=1
                        if value==0:
                            pyautogui.click(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
                            print(x,y)
                            isfind=True
                            break
                if isfind:
                    break
            if isfind:
                break

In [48]:
def game_auto_clear2(remain_mine,find_mine_num):
    global remain_blank
    time.sleep(WAIT)
    buttonlocation = pyautogui.locateOnScreen('start.png')
    startpoint=pyautogui.center(buttonlocation)
    start_x=startpoint.x-BOX_LEN*((WIDTH-1)/2)
    start_y=startpoint.y+52
    screen = ImageGrab.grab()
    sx=-1
    sy=-1

    #initalize mine_map array
    for x in range(WIDTH):
        for y in range(HEIGHT):
            check_point=pyautogui.Point(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
            mine_map[y][x] = check_color(check_point,screen)
            if mine_map[y][x] is None:
                sx=x
                sy=y

    if sx!=-1 and sy!=-1:
        pyautogui.click(start_x+BOX_LEN*sx,start_y+BOX_LEN*sy)
        

    for l in range(find_mine_num):
        time.sleep(WAIT)
        #print("remain_mine "+str(remain_mine))
        #print("remain_blank "+str(remain_blank))
        screen = ImageGrab.grab()
        for x in range(WIDTH):
            for y in range(HEIGHT):
                if mine_map[y][x]==0 or mine_map[y][x]==9:
                    check_point=pyautogui.Point(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
                    mine_map[y][x] = check_color(check_point,screen)
                    if mine_map[y][x] is None:
                        mine_map[y][x]=9
        remain_blank=0                
        for x in range(WIDTH):
            for y in range(HEIGHT):
                if mine_map[y][x]==0:
                    remain_blank+=1
                    
        if remain_blank==0:
            return
        
        isfind=False
        
        for x in range(0,WIDTH):
            for y in range(0,HEIGHT):
                if mine_map[y][x]!=0:
                    isclick=False
                    sum_val=0
                    x_range,y_range = range_list(x,y,1)
                    for i in x_range:
                        for j in y_range:
                            if mine_map[j][i]==-1:
                                sum_val+=1
                            if mine_map[j][i]==0:
                                isclick=True
                    if sum_val==mine_map[y][x] and isclick:
                        pyautogui.click(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
                        isfind=True
                        break
            if isfind:
                break
        if isfind:
            continue
        
        
        isfind=False
        for k in range(1,6,2):
            for x in range(0,WIDTH):
                for y in range(0,HEIGHT):
                    if mine_map[y][x]==0:
                        value = find_mine(x,y,k,remain_mine)
                        if value==1:
                            mine_map[y][x]=-1
                            pyautogui.rightClick(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
                            remain_mine-=1
                        if value==0:
                            pyautogui.click(start_x+BOX_LEN*x,start_y+BOX_LEN*y)
                            isfind=True
                            break
                if isfind:
                    break
            if isfind:
                break
        #if not isfind:
        #    return

In [49]:
#inital setting

BOX_LEN=24
WIDTH=30
HEIGHT=20
mine_map=list()

COLOR={(198, 198, 198):0,
       (0, 0, 255):1,
       (112, 168, 112):2,
       (50,146,50):2,
       (255, 0, 0):3,
       (0, 0, 128):4,
       (128, 0, 0):5,
       (0, 128, 128):6,
      (0,0,0):7,
      (96,78,78):-1} 


remain_blank=WIDTH*HEIGHT

for y in range(HEIGHT):
    mine_map.append([-1 for x in range(WIDTH)])
mine_map[0][1]=1

In [51]:
WAIT=0
remain_mine=130
game_auto_clear(remain_mine,WIDTH*HEIGHT)
#    pyautogui.click(buttonlocation)

remain_mine 130
remain_blank 600
4 12
remain_mine 130
remain_blank 544
3 13
remain_mine 130
remain_blank 543
3 15
remain_mine 129
remain_blank 542
3 11
remain_mine 129
remain_blank 540
4 11
remain_mine 128
remain_blank 539
3 10
remain_mine 128
remain_blank 537
2 9
remain_mine 128
remain_blank 536
2 10
remain_mine 128
remain_blank 535
1 12
remain_mine 127
remain_blank 529
0 8
remain_mine 127
remain_blank 527
0 9
remain_mine 127
remain_blank 526
0 10
remain_mine 127
remain_blank 525
2 8
remain_mine 126
remain_blank 524
2 13
remain_mine 126
remain_blank 522
2 14
remain_mine 126
remain_blank 521
3 8
remain_mine 126
remain_blank 520
4 8
remain_mine 126
remain_blank 519
4 9
remain_mine 126
remain_blank 518
9 7
remain_mine 114
remain_blank 517
2 7
remain_mine 114
remain_blank 504
1 6
remain_mine 114
remain_blank 503
2 6
remain_mine 114
remain_blank 502
3 6
remain_mine 113
remain_blank 501
0 12
remain_mine 113
remain_blank 499
3 12
remain_mine 113
remain_blank 498
1 15
remain_mine 113
remain_b